In [1]:
#%pip install beautifulsoup4 lxml html5lib
#%pip install EbookLib

In [2]:
from zipfile import ZipFile
import glob

In [3]:
import warnings
from bs4 import MarkupResemblesLocatorWarning, XMLParsedAsHTMLWarning, BeautifulSoup
import bs4
#warnings.filterwarnings("ignore", category=XMLParsedAsHTMLWarning)

In [4]:


#def extractEpub(filename: str):
#  archive = ZipFile(f'input/{filename}')
#  archive.extractall(f"data/{filename}")
#  archive.close()

In [5]:
#extractEpub('Spice and Wolf, Vol. 9.epub')

In [6]:

inputEpubs = glob.glob('*', root_dir='./input')

In [7]:
def readAll(filename: str):
  file = open(filename, encoding='utf-8')
  text = file.read()
  file.close()
  return text

def writeAll(filename: str, text: str):
  file = open(filename, encoding='utf-8', mode='w')
  text = file.write(text)
  return text


In [8]:
from bs4 import BeautifulSoup
import os


#for volume in dataFolders:
#  htmlFiles = glob.glob(f"{volume}\\*.html")
#
#  testFile = htmlFiles[0]
#  #print(testFile)
#  hehe = BeautifulSoup(readAll(testFile), "html5lib").contents[1]
#
#  writeAll(f'{volume}/testoutput.html', hehe.prettify())
#  paragraphs = hehe.find_next('body')
#  for v in paragraphs:
#   print(v)
#
#  #for v in hehe:
#  #  #print(v)
#  #  print('helo')
#  ##print()




In [9]:
from typing import List

from bs4 import NavigableString, Tag

def calculateIndex(value: int):
  #if value < 4:
  #  return value // 2 + 1
  #if value == 1:
  #  return 1
  #if value == 2:
  #  return 1
  #if value == 3:
  #  return 1
  #if value == 4:
  #  return 2
  #if value == 5:
  #  return 2
  #if value == 6:
  #  return 3
  return value // 3 + 1


def parseText(soup: BeautifulSoup, text: str):
  def embolden(t: str):
    dividerIndex = len(t) // 4 + 1
    boldPart = soup.new_tag('b')
    boldPart.string = t[0: dividerIndex]

    return [boldPart, soup.new_string(t[dividerIndex:])]

  result : List[Tag | NavigableString] = []

  startI = 0
  while startI < len(text):
    char = text[startI]
    if not char.isalpha():
      result.append(char)
      startI += 1
      continue;

    if char == '\'' or char == '"':
      result.extend(embolden(char))
      startI += 1
      continue;

    endI = startI
    while endI < len(text) and text[endI].isalpha():
      endI += 1

    word = text[startI: endI]

    result.extend(embolden(word))
    startI = endI
  return result

#result = parseText(BeautifulSoup(), 'hello how are you? ?e')
#print(result)

In [10]:
from typing import Iterable, Iterator

from bs4 import PageElement

def skip(iter: Iterator, count: int):
  for i in range(0, count):
    try:
      iter.__next__()
    except StopIteration:
      return

def parseTree(soup: BeautifulSoup, elements: Iterable[PageElement]):
  iter = elements.__iter__()
  while True:
    try:
      next = iter.__next__()

      if next.string is None:
        parseTree(soup, next.children)
      else:
        boldText = parseText(soup, next.string)
        next.replace_with(*boldText)
        skip(iter, len(boldText) - 1)

    except StopIteration:
      break;





In [11]:
from typing import Iterator


def cleanupText(text: str):
  return text.replace('\n', '')

#page = BeautifulSoup(cleanupText("""
#<html xmlns="http://www.w3.org/1999/xhtml" lang="" xml:lang="">
#<body>
#hello there
#<p class="calibre1"><a id="p10"></a><a href="#p12">Copyright</a></p>
#<p class="calibre1">SPICE AND WOLF, Volume 9: The Town of Strife II</p>
#</body>
#</html>
#                     """), "html5lib")


#paths = ['./data/Spice and Wolf, Vol. 9.epub/index_split_000.html', './data/Spice and Wolf, Vol. 9.epub/index_split_001.html', './data/Spice and Wolf, Vol. 9.epub/index_split_002.html']
#
#for path in paths:
#  page = BeautifulSoup(readAll(path), 'html5lib')
#
#
#  iter = page.children.__iter__()
#
#  parseTree(page, page.children)
#
#  #print(page.prettify())
#
#  writeAll(path, str(page))


In [12]:

import ebooklib
from ebooklib import epub
from ebooklib.epub import EpubHtml


def convertBook(filename: str):
  book = epub.read_epub(f'./input/{filename}')
  book.set_title(book.title + 'bold')
  print(book.title)

  for document in book.get_items_of_type(ebooklib.ITEM_DOCUMENT):
    print(type(document))
    document : EpubHtml = document
    documentStr = str(document.content, encoding='utf-8')
    parsedDocument = BeautifulSoup(documentStr, 'xml')
    parseTree(parsedDocument, parsedDocument.children)
    document.content = bytes(str(parsedDocument), 'utf-8')
    #print(type(document.content))

  epub.write_epub(f'./output/bold-{filename}', book)



In [13]:
for book in inputEpubs:
	#print(book)
  convertBook(book)

c:\Users\Andrii\.vscode\code\python\book-bold-processor\.venv\lib\site-packages\ebooklib\epub.py:1395: UserWarning: In the future version we will turn default option ignore_ncx to True.
  warnings.warn('In the future version we will turn default option ignore_ncx to True.')
c:\Users\Andrii\.vscode\code\python\book-bold-processor\.venv\lib\site-packages\ebooklib\epub.py:1423: FutureWarning: This search incorrectly ignores the root element, and will be fixed in a future version.  If you rely on the current behaviour, change it to './/xmlns:rootfile[@media-type]'
  for root_file in tree.findall('//xmlns:rootfile[@media-type]', namespaces={'xmlns': NAMESPACES['CONTAINERNS']}):


Великий Гетсбіbold
<class 'ebooklib.epub.EpubHtml'>
<class 'ebooklib.epub.EpubHtml'>
<class 'ebooklib.epub.EpubHtml'>
<class 'ebooklib.epub.EpubHtml'>


In [ ]:





#writeAll(f'{volume}/testoutput.html', page.prettify())
#paragraphs = hehe.find_next('body')
#print(list(paragraphs.descendants))
#print(paragraphs.descendants)
#iterator = paragraphs.__iter__()
#print(list(paragraphs.children))
#hello = iterator.__next__()
##hello = list(paragraphs.children)[0]
#b1 = hehe.new_tag('b')
#b1.string = hello.text + '1'
#
#b2 = hehe.new_tag('b')
#b2.string = hello.text + '2'
#
#hello.replaceWith(*[b1, b2])
#next = iterator.__next__()
#print(type(next))
#if next.name != 'b':
#  print('keke')
#paragraphs.
#print(list(paragraphs.children))
#for v in paragraphs:
#  print(v.text)
#  print(v)

#for v in hehe:
#  #print(v)
#  print('helo')
##print()

('hello', 'there')


In [35]:

import ebooklib
from ebooklib import epub
from ebooklib.epub import EpubHtml
import re


def testBook(filename: str):
  book = epub.read_epub(f'./input/{filename}')
  book.set_title(book.title + 'bold')
  print(book.title)

  document = list(book.get_items_of_type(ebooklib.ITEM_DOCUMENT))[1]
  print(type(document))
  document : EpubHtml = document
  documentStr = str(document.get_body_content(), encoding='utf-8')
  parsedDocument = BeautifulSoup(documentStr, 'xml')

  parseTree(parsedDocument, parsedDocument.children)
  text = parsedDocument.prettify()

  #pattern = re.compile("<.+>")
  #result = pattern.match(text)
  skippedText = "<?xml version=\"1.0\" encoding=\"utf-8\"?>"
  result = text.find(skippedText)
  if result != -1:
    text = text[result + len(skippedText):]
  document.set_content(bytes(text, 'utf-8'))

  #epub.write_epub(f'./output/test-{filename}', book)



testBook('Spice and Wolf, Vol. 10.epub')



#soup = BeautifulSoup('''
#<?xml version='1.0' encoding='utf-8'?>
#<html xmlns="http://www.w3.org/1999/xhtml" lang="" xml:lang="">
#  <head>
#    <title>Spice and Wolf, Vol. 10</title>
#    <meta name="generator" content="pdftohtml 0.36"/>
#    <meta name="author" content="Isuna Hasekura &amp; Jyuu Ayakura"/>
#    <meta name="date" content="2023-01-23T17:57:18+00:00"/>
#    <meta http-equiv="Content-Type" content="text/html; charset=utf-8"/>
#  <link rel="stylesheet" type="text/css" href="stylesheet.css"/>
#<link rel="stylesheet" type="text/css" href="page_styles.css"/>
#</head>
#  <body class="calibre">
#  </body>
#  </html>
#                     ''', 'xml')
#
#
#print(soup)

Spice and Wolf, Vol. 10bold
<class 'ebooklib.epub.EpubHtml'>
